In [1]:
import pandas as pd

## 处理北京天气

In [2]:
import pandas as pd

# ========== 1. 定义 ISD-Lite 每列的字段信息 ==========
# Define fixed-width field specifications (start, end, column name)
colspecs = [
    (0, 4),   # Year
    (5, 7),   # Month
    (8, 10),  # Day
    (11, 13), # Hour
    (13, 19), # Air temperature
    (19, 25), # Dew point temperature
    (25, 31), # Sea level pressure
    (31, 37), # Wind direction
    (37, 43), # Wind speed
    (43, 49), # Sky condition
    (49, 55), # Precipitation 1-hour
    (55, 61), # Precipitation 6-hour
]

colnames = [
    "year", "month", "day", "hour",
    "air_temp", "dew_point", "sea_level_pressure",
    "wind_dir", "wind_speed", "sky_condition",
    "precip_1h", "precip_6h"
]

# ========== 2. 读取 ISD-Lite 文件 ==========
# Replace 'sample.txt' with your file path
file_path = "545110-99999-2017.csv"
df = pd.read_fwf(file_path, colspecs=colspecs, names=colnames)

# ========== 3. 处理缩放因子（Scaling factor） ==========
# Apply scaling factors
scale_10_cols = ["air_temp", "dew_point", "sea_level_pressure", 
                 "wind_speed", "precip_1h", "precip_6h"]

for col in colnames:
    df[col] = df[col].replace(-9999, pd.NA)   # 替换缺失值
    # 转为int
    df[col] = df[col].astype("Int64")

for col in scale_10_cols:
    df[col] = df[col] / 10.0                  # 除以10，得到实际值


# precip_xh (降水量) 特殊处理：-1 表示痕量降水 trace precipitation
for col in ["precip_1h", "precip_6h"]:
    df[col] = df[col].replace(-0.1, 0.1)
    # 或者如果你想保留为 0 mm，可以改为：
    # df[col] = df[col].replace(-0.1, 0.0)

# ========== 4. 添加一个完整的时间列 ==========
df["datetime"] = pd.to_datetime(df[["year", "month", "day", "hour"]], errors='coerce')
df = df.set_index("datetime")       # 将 datetime 设为 index
df = df.drop(columns=["year", "month", "day", "hour"])   # 删除原始时间列
# ========== 5. 输出处理后的数据 ==========
df.head(50)


,air_temp,dew_point,sea_level_pressure,wind_dir,wind_speed,sky_condition,precip_1h,precip_6h
datetime,,,,,,,,
2017-01-01 00:00:00,-4.9,-5.7,1026.9,70,1.0,8,<NA>,<NA>
2017-01-01 01:00:00,-6.0,-7.0,<NA>,<NA>,1.0,0,<NA>,<NA>
2017-01-01 02:00:00,-5.0,-6.0,<NA>,<NA>,1.0,0,<NA>,<NA>
2017-01-01 03:00:00,-3.7,-4.5,1026.7,200,1.0,8,<NA>,<NA>
2017-01-01 04:00:00,-2.0,-5.0,<NA>,150,2.0,0,<NA>,<NA>
2017-01-01 05:00:00,0.0,-6.0,<NA>,<NA>,1.0,0,<NA>,<NA>
2017-01-01 06:00:00,-1.7,-4.7,1024.2,200,2.0,7,<NA>,0.1
2017-01-01 07:00:00,-1.0,-4.0,<NA>,150,2.0,0,<NA>,<NA>
2017-01-01 08:00:00,-1.0,-4.0,<NA>,140,2.0,0,<NA>,<NA>


In [3]:
# 计算每列的统计信息
stats = df.describe()
stats

,air_temp,dew_point,sea_level_pressure,wind_dir,wind_speed,sky_condition,precip_1h,precip_6h
count,8733.0,8731.0,2914.0,7493.0,8733.0,4338.0,0.0,130.0
mean,13.791526,2.337785,1016.998902,178.776191,2.717279,4.370447,<NA>,4.467692
std,11.781191,14.079922,10.420956,113.248682,2.044314,3.955998,<NA>,9.48918
min,-13.0,-28.0,994.0,0.0,0.0,0.0,<NA>,0.0
25%,3.4,-10.0,1008.1,80.0,1.0,0.0,<NA>,0.1
50%,14.6,3.0,1017.1,180.0,2.0,7.0,<NA>,0.9
75%,24.0,15.0,1025.5,290.0,3.0,9.0,<NA>,4.0
max,39.0,28.0,1040.2,360.0,16.0,9.0,<NA>,66.0


In [4]:
# 丢弃wind_dir 列
df = df.drop(columns=["wind_dir"])
df = df.drop(columns=["precip_1h", "precip_6h"])  # 降雨数据缺失率太高

# 对于sky_condition 列，每个非0值后面都是空值，但是被计作了0，要恢复
sky_cond = df['sky_condition']
i = 0
while i < len(sky_cond) - 2:
    if sky_cond.iloc[i] is not pd.NA:
        if sky_cond.iloc[i] != 0:
            sky_cond.iloc[i+1] = pd.NA
            sky_cond.iloc[i+2] = pd.NA
    i += 3

df['sky_condition'] = sky_cond
       

#剩余列确实值使用线性插值填补
df = df.interpolate(method='linear')
# 输出处理后的数据
df.head(50)


,air_temp,dew_point,sea_level_pressure,wind_speed,sky_condition
datetime,,,,,
2017-01-01 00:00:00,-4.9,-5.7,1026.9,1.0,8.0
2017-01-01 01:00:00,-6.0,-7.0,1026.833333,1.0,8.0
2017-01-01 02:00:00,-5.0,-6.0,1026.766667,1.0,8.0
2017-01-01 03:00:00,-3.7,-4.5,1026.7,1.0,8.0
2017-01-01 04:00:00,-2.0,-5.0,1025.866667,2.0,7.666667
2017-01-01 05:00:00,0.0,-6.0,1025.033333,1.0,7.333333
2017-01-01 06:00:00,-1.7,-4.7,1024.2,2.0,7.0
2017-01-01 07:00:00,-1.0,-4.0,1023.966667,2.0,7.0
2017-01-01 08:00:00,-1.0,-4.0,1023.733333,2.0,7.0


In [5]:
stats = df.describe()
stats

,air_temp,dew_point,sea_level_pressure,wind_speed,sky_condition
count,8733.0,8733.0,8733.0,8733.0,8733.0
mean,13.791526,2.335498,1016.988601,2.717279,5.902668
std,11.781191,14.079122,10.39962,2.044314,3.267818
min,-13.0,-28.0,994.0,0.0,0.0
25%,3.4,-10.0,1008.1,1.0,3.333333
50%,14.6,3.0,1017.0,2.0,7.0
75%,24.0,15.0,1025.566667,3.0,9.0
max,39.0,28.0,1040.2,16.0,9.0


In [6]:
# 对每一列都归一化
for col in df.columns:
    df[col] = (df[col] - df[col].min()) / (df[col].max() - df[col].min())
        

In [7]:
df

,air_temp,dew_point,sea_level_pressure,wind_speed,sky_condition
datetime,,,,,
2017-01-01 00:00:00,0.155769,0.398214,0.712121,0.0625,0.888889
2017-01-01 01:00:00,0.134615,0.375,0.710678,0.0625,0.888889
2017-01-01 02:00:00,0.153846,0.392857,0.709235,0.0625,0.888889
2017-01-01 03:00:00,0.178846,0.419643,0.707792,0.0625,0.888889
2017-01-01 04:00:00,0.211538,0.410714,0.689755,0.125,0.851852
...,...,...,...,...,...
2017-12-31 19:00:00,0.096154,0.232143,0.7114,0.125,1.0
2017-12-31 20:00:00,0.076923,0.232143,0.712843,0.1875,1.0
2017-12-31 21:00:00,0.15,0.221429,0.714286,0.0625,1.0


In [8]:
need_time = pd.read_parquet('beijing_dataset_df_15min.parquet').index
# 将 df 的索引对齐到 need_time
df = df.reindex(need_time, method='nearest')

In [9]:
# 保存到 CSV
df.to_csv("beijing_weather_normalized.csv", index=True)


## 处理NYC天气

In [10]:
nyc_df = pd.read_csv("NYC_Weather_2016_2022.csv")
nyc_df.head(10)

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
0,2016-01-01T00:00,7.6,0.0,0.0,69.0,53.0,0.0,72.0,10.0,296.0
1,2016-01-01T01:00,7.5,0.0,0.0,20.0,4.0,0.0,56.0,9.8,287.0
2,2016-01-01T02:00,7.1,0.0,0.0,32.0,3.0,0.0,99.0,9.7,285.0
3,2016-01-01T03:00,6.6,0.0,0.0,35.0,5.0,0.0,100.0,9.2,281.0
4,2016-01-01T04:00,6.3,0.0,0.0,34.0,4.0,0.0,100.0,9.1,279.0
5,2016-01-01T05:00,6.1,0.0,0.0,35.0,5.0,0.0,100.0,9.4,277.0
6,2016-01-01T06:00,6.0,0.0,0.0,50.0,21.0,1.0,100.0,9.7,274.0
7,2016-01-01T07:00,5.9,0.0,0.0,51.0,24.0,0.0,98.0,9.7,272.0
8,2016-01-01T08:00,5.8,0.0,0.0,54.0,26.0,1.0,99.0,9.0,265.0
9,2016-01-01T09:00,5.8,0.0,0.0,58.0,31.0,1.0,99.0,10.2,262.0


In [11]:
# 只保留2019-2 到 2019-5的数据
nyc_df['time'] = pd.to_datetime(nyc_df['time'])
nyc_df = nyc_df[(nyc_df['time'] >= '2019-02-01') & (nyc_df['time'] < '2019-06-01')]
nyc_df.describe()

,time,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h),winddirection_10m (°)
count,2880,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000,2880.000000
mean,2019-04-01 23:30:00,8.747396,0.136146,0.118785,50.813194,28.519792,36.114583,47.982639,12.078056,201.423611
min,2019-02-01 00:00:00,-11.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.400000,2.000000
25%,2019-03-02 23:45:00,2.700000,0.000000,0.000000,12.000000,0.000000,0.000000,0.000000,7.600000,115.000000
50%,2019-04-01 23:30:00,8.600000,0.000000,0.000000,45.000000,2.000000,16.000000,47.000000,11.100000,209.000000
75%,2019-05-01 23:15:00,14.800000,0.000000,0.000000,100.000000,61.000000,81.000000,97.000000,15.200000,291.000000
max,2019-05-31 23:00:00,28.400000,4.900000,4.900000,100.000000,100.000000,100.000000,100.000000,41.300000,360.000000
std,NaN,7.591503,0.445114,0.417453,39.068138,39.403722,39.870780,43.732273,6.078175,101.061451


In [12]:
# 归一化
nyc_df.index = nyc_df['time']  # 将时间列设为索引
nyc_df = nyc_df.drop(columns=['time'])  # 删除原时间列
nyc_df = nyc_df.drop(columns=['winddirection_10m (°)'])
for col in nyc_df.columns:
    nyc_df[col] = (nyc_df[col] - nyc_df[col].min()) / (nyc_df[col].max() - nyc_df[col].min())

In [13]:
nyc_df.head()

,temperature_2m (°C),precipitation (mm),rain (mm),cloudcover (%),cloudcover_low (%),cloudcover_mid (%),cloudcover_high (%),windspeed_10m (km/h)
time,,,,,,,,
2019-02-01 00:00:00,0.0525,0.0,0.0,0.0,0.0,0.0,0.0,0.317848
2019-02-01 01:00:00,0.0525,0.0,0.0,0.0,0.0,0.0,0.0,0.305623
2019-02-01 02:00:00,0.0450,0.0,0.0,0.0,0.0,0.0,0.0,0.251834
2019-02-01 03:00:00,0.0300,0.0,0.0,0.0,0.0,0.0,0.0,0.200489
2019-02-01 04:00:00,0.0250,0.0,0.0,0.0,0.0,0.0,0.0,0.166259


In [14]:
need_time = pd.read_parquet('nyc_dataset_df_15min.parquet').index
# 将 df 的索引对齐到 need_time
nyc_df = nyc_df.reindex(need_time, method='nearest')

In [15]:
nyc_df.to_csv("nyc_weather_normalized.csv", index=True)

In [19]:
print(nyc_df.reset_index(drop=False).values)
print(type(nyc_df.reset_index(drop=False).values))

[[Timestamp('2019-02-01 06:00:00') 0.019999999999999973 0.0 ... 0.01 0.0
  0.1784841075794621]
 [Timestamp('2019-02-01 06:15:00') 0.019999999999999973 0.0 ... 0.01 0.0
  0.1784841075794621]
 [Timestamp('2019-02-01 06:30:00') 0.02749999999999999 0.0 ... 0.0 0.0
  0.16625916870415647]
 ...
 [Timestamp('2019-05-31 23:15:00') 0.8949999999999999 0.0 ... 0.3 0.0
  0.11491442542787285]
 [Timestamp('2019-05-31 23:30:00') 0.8949999999999999 0.0 ... 0.3 0.0
  0.11491442542787285]
 [Timestamp('2019-05-31 23:45:00') 0.8949999999999999 0.0 ... 0.3 0.0
  0.11491442542787285]]
<class 'numpy.ndarray'>
